In [117]:
from sklearn.model_selection import RepeatedStratifiedKFold,GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel,VarianceThreshold
import numpy as np
import pandas as pd
from pprint import pprint
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import StratifiedKFold
from scipy import stats
import numpy as np
from sklearn import linear_model, svm
import re
from sklearn.metrics import roc_curve, auc,f1_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
#s = "../braindata/data_1_mor_select_100.csv"
import os          
from sklearn import linear_model, svm
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC


import pandas as pd
import numpy as np
import logging
import time
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV



os.getcwd()
os.chdir('/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/idp_jiook/data/adni')
dd =pd.read_csv("combine_new_biomarker.csv",header=0)
dd.info()
print(dd.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Columns: 16384 entries, RID to T16360
dtypes: float64(3496), int64(12888)
memory usage: 22.4 MB
(179, 16384)


In [118]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# corr_matrix=dd.corr()
# from pandas.tools.plotting import scatter_matrix
# att=["finaldx","dxbl","Agebl","ABETA","TAU","PTAU","ratio1","ratio2","PTGENDER"]
# scatter_matrix(dd[att],figsize=(20,15))
# #dd.hist(bins=3,figsize=(20,15))
# dd.plot(kind='scatter',x='dxbl',y='ratio2')
# dd.corr
# plt.show()


with open('combine_new_biomarker.csv', 'r') as f:
    d_reader = csv.DictReader(f)

    #get fieldnames from DictReader object and store in list
    headers = d_reader.fieldnames

data=np.array(dd)
print(data.shape)
print(data[0])

#idx_IN_columns = np.append(np.array([10,18]),np.array(range(22,data.shape[1])))
idx_IN_columns = np.array([19,20,21,22,23])

print(idx_IN_columns)
X=data[:,idx_IN_columns]
import matplotlib.pyplot as plt
plt.show()
#features=data[:,11:data.shape[1]]
#features = features.transpose()
#X = stats.zscore(X)
#print(features.shape)
y=data[:,2]
print(y)
#/ 6:AD-normal / 7:AD-MCI / 8:MCI-normal 

ind_num=np.isnan(y)
# print(ind_num.shape)


y_no_nan = y[~ind_num]

X_no_nan = X[~ind_num,:]

       # print(y.shape)

y=y_no_nan
X=X_no_nan
feature_num_all=[]
lr_all_feature=[]
svm_all_feature=[]
lr_fls_feature=[]
svm_fls_feature=[]
base_labels= []

np.isnan(X).any()

X[np.isnan(X)] = np.median(X[~np.isnan(X)])
print(X.shape)
print(y.shape)

(179, 16384)
[  4.08100000e+03   1.00000000e+00              nan ...,   1.50000000e+01
   5.00000000e+00   3.00000000e+00]
[19 20 21 22 23]
[ nan  nan  nan  nan  nan  nan  nan   0.  nan  nan   0.  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan   0.   0.  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan   0.  nan   1.  nan   0.   0.
  nan  nan   1.  nan  nan   0.   1.  nan  nan  nan   1.  nan  nan   0.  nan
   1.  nan   1.   1.  nan  nan   0.   0.   0.   1.  nan   1.  nan   0.  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan   0.  nan  nan  nan  nan
   0.  nan  nan  nan   0.   1.   0.   0.   1.  nan  nan   1.   0.   0.  nan
  nan  nan  nan   1.  nan  nan   1.  nan  nan   0.  nan  nan  nan  nan   0.
  nan  nan  nan  nan  nan  nan   0.   0.   0.   0.  nan   0.   0.   1.  nan
   1.   0.   1.  nan   1.   0.   0.   0.  nan  nan  nan   0.  nan   0.   1.
   0.  nan  nan   0.  nan  nan   1.  nan   1.   1.  nan  nan  nan  nan  nan
  nan  nan  nan  nan  na

In [119]:
selector = VarianceThreshold()
selector.fit_transform(X)
print(X.shape)
features=X

# model = RandomForestClassifier()
# parameters = [{"n_estimators": [50,100,250, 500, 1000,5000]}]
 
# Returns the best configuration for a model using crosvalidation
# and grid search
# def best_config(model, parameters, train_instances, judgements):
#     clf = GridSearchCV(model, parameters, cv=5,
#                        scoring="accuracy", verbose=5, n_jobs=4)
#     clf.fit(train_instances, judgements)
#     best_estimator = clf.best_estimator_
#     #log_info('Best hyperparameters: ' + str(clf.best_params_))
#     return [str(clf.best_params_), clf.best_score_, best_estimator]


# best_config(model,parameters,X,y)
lr_all_accs = []
lr_all_scores = []
lr_f1s = []
svm_all_accs = []
svm_all_scores = []
svm_f1s = []

lr_accuracies = []
lr_scores = []
svm_accuracies = []
svm_scores = []
strat_labels = []
rf_all_accs = []
rf_all_scores = []
rf_f1s = []

rf_accuracies = []
rf_scores = []
logistic = linear_model.LogisticRegression(C=1e5)
rbf = svm.SVC(C=10,kernel='linear',gamma=0.001)
rf = ExtraTreesClassifier(n_estimators=500, random_state=0)


skf=RepeatedStratifiedKFold(n_splits=10, n_repeats=2)
for train_index, test_index in skf.split(features, y):
        train_data, test_data = features[train_index], features[test_index]
        train_labels, test_labels = y[train_index], y[test_index]

        strat_labels = np.append(strat_labels, test_labels)

        log = logistic.fit(train_data, train_labels)
        log_prob = log.decision_function(test_data)
        log_acc = log.score(test_data, test_labels)

        #f1 calculation
        y_pred = logistic.predict(test_data)
        log_f1 = f1_score(test_labels, y_pred)

        lr_accuracies = np.append(lr_accuracies, log_acc)
        lr_scores = np.append(lr_scores, log_prob)
        lr_f1s = np.append(lr_f1s, log_f1)
        #print('Logistic Regression Accuracy: %f' % log_acc)

        rbf = rbf.fit(train_data, train_labels)
        svm_acc = rbf.score(test_data, test_labels)
        svm_prob = rbf.decision_function(test_data)

        #f1 calculation
        y_pred = rbf.predict(test_data)
        svm_f1 = f1_score(test_labels, y_pred)
        #print('SVM Accuracy: %f' % svm_acc)
        svm_accuracies = np.append(svm_accuracies, svm_acc)
        svm_scores = np.append(svm_scores, svm_prob)
        svm_f1s = np.append(svm_f1s, svm_f1)
        #print('SVM f1: %f' % svm_f1)

        rf = rf.fit(train_data, train_labels)
        rf_acc = rf.score(test_data, test_labels)
       # rf_prob = rf.decision_function(test_data)

#         #f1 calculation
        y_pred = rf.predict(test_data)
        rf_f1 = f1_score(test_labels, y_pred)
       # print('Tree Accuracy: %f' % rf_acc)
        rf_accuracies = np.append(rf_accuracies, rf_acc)
        #rf_scores = np.append(rf_scores, rf_prob)
        rf_f1s = np.append(rf_f1s, rf_f1)
        #print('SVM f1: %f' % svm_f1)

        lr_all_scores = np.append(lr_all_scores, lr_scores)
        lr_fold_avg = np.mean(lr_accuracies)
        lr_all_accs = np.append(lr_all_accs, lr_fold_avg)
        svm_all_scores = np.append(svm_all_scores, svm_scores)
        svm_fold_avg = np.mean(svm_accuracies)
        svm_all_accs = np.append(svm_all_accs, svm_fold_avg)
        svm_all_accs = np.append(svm_all_accs, svm_fold_avg)

        #rf_all_scores = np.append(rf_all_scores, rf_scores)
        rf_fold_avg = np.mean(rf_accuracies)
        rf_all_accs = np.append(rf_all_accs, rf_fold_avg)
        #print('Logistic Regression Accuracy: %f' % lr_acc_avg)
        #print('SVM Regression Accuracy: %f' % svm_acc_avg)
#  feature_num_all=np.append(feature_num_all,feature_num)
print(np.mean(lr_all_accs))
print(np.mean(svm_all_accs))     
print(np.mean(rf_all_accs))      

#         lr_all_feature=np.append(lr_all_feature,np.mean(lr_all_accs))
#         svm_all_feature=np.append(svm_all_feature,np.mean(svm_all_accs))
#         lr_fls_feature=np.append(lr_fls_feature,np.mean(lr_f1s))
#         svm_fls_feature=np.append(svm_fls_feature,np.mean(svm_f1s))
#         #base_labels_all = np.append(base_labels_all,strat)
#         base_labels = np.append(base_labels, np.mean(strat_labels))

(60, 5)


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.70891640867
0.660667007535
0.630253825615


In [104]:
model = RandomForestClassifier()
parameters = [{"n_estimators": [50,100,250, 500, 1000]}]
 
def best_config(model, parameters, train_instances, judgements):
    #log_info('Grid search for... ' + name)
    clf = GridSearchCV(model, parameters, cv=10,
                       scoring="accuracy", verbose=5, n_jobs=4)
    clf.fit(train_instances, judgements)
    best_estimator = clf.best_estimator_
    #log_info('Best hyperparameters: ' + str(clf.best_params_))
 
    return [str(clf.best_params_), clf.best_score_,
            best_estimator]
best_config(model,parameters,X,y)

Fitting 10 folds for each of 5 candidates, totalling 50 fits
[CV] n_estimators=50 .................................................
[CV] n_estimators=50 .................................................
[CV] n_estimators=50 .................................................
[CV] n_estimators=50 .................................................
[CV] ........................ n_estimators=50, score=0.473684 -   0.1s
[CV] n_estimators=50 .................................................
[CV] ........................ n_estimators=50, score=0.611111 -   0.1s
[CV] ........................ n_estimators=50, score=0.444444 -   0.1s
[CV] n_estimators=50 .................................................
[CV] n_estimators=50 .................................................
[CV] ........................ n_estimators=50, score=0.111111 -   0.1s
[CV] n_estimators=50 .................................................
[CV] ........................ n_estimators=50, score=0.388889 -   0.1s
[CV] ...........

[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:    0.5s


[CV] ....................... n_estimators=100, score=0.411765 -   0.2s
[CV] n_estimators=250 ................................................
[CV] ....................... n_estimators=250, score=0.222222 -   0.4s
[CV] ....................... n_estimators=250, score=0.473684 -   0.4s
[CV] n_estimators=250 ................................................
[CV] n_estimators=250 ................................................
[CV] ....................... n_estimators=250, score=0.444444 -   0.4s
[CV] n_estimators=250 ................................................
[CV] ....................... n_estimators=250, score=0.722222 -   0.4s
[CV] n_estimators=250 ................................................
[CV] ....................... n_estimators=250, score=0.666667 -   0.3s
[CV] n_estimators=250 ................................................
[CV] ....................... n_estimators=250, score=0.500000 -   0.4s
[CV] n_estimators=500 ................................................
[CV] .

[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    6.9s finished


["{'n_estimators': 250}",
 0.4748603351955307,
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False)]

In [99]:
from sklearn import svm, grid_search
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_


svc_param_selection(X,y,10)

{'C': 10, 'gamma': 0.001}

In [98]:
# Returns the best model from a set of model families given
# training data using cross-validation.
def best_config(model, parameters, train_instances, judgements):
    #log_info('Grid search for... ' + name)
    clf = GridSearchCV(model, parameters, cv=5,
                       scoring="accuracy", verbose=5, n_jobs=4)
    clf.fit(train_instances, judgements)
    best_estimator = clf.best_estimator_
    #log_info('Best hyperparameters: ' + str(clf.best_params_))
 
    return [str(clf.best_params_), clf.best_score_,
            best_estimator]


def best_model(classifier_families, train_instances, judgements):
    best_quality = 0.9
    best_classifier = None
    classifiers = []
    for name, model, parameters in classifier_families:
        classifiers.append(best_config(model, parameters,
                                       train_instances,
                                       judgements))
 
    for name, quality, classifier in classifiers:
        if (quality > best_quality):
            
            best_quality = quality
            best_classifier = [name, classifier]
 
    return best_classifier


# List of candidate family classifiers with parameters for grid
# search [name, classifier object, parameters].
def candidate_families():
    candidates = []
    svm_tuned_parameters = [{'kernel': ['poly'],
                             'degree': [1, 2]}]
    candidates.append(["SVM", SVC(C=10), svm_tuned_parameters])
    
    rf_tuned_parameters = [{"n_estimators": [50, 100,250, 500, 1000]}]
    candidates.append(["RandomForest",
                       RandomForestClassifier(n_jobs=-1),
                       rf_tuned_parameters])
 #     knn_tuned_parameters = [{"n_neighbors": [1, 3, 5, 10, 20]}]
#     candidates.append(["kNN", KNeighborsClassifier(),
#                        knn_tuned_parameters])
 
    return candidates


candidates=candidate_families()
best_model(candidates,X,y)


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] degree=1, kernel=poly ...........................................
[CV] degree=1, kernel=poly ...........................................
[CV] degree=1, kernel=poly ...........................................
[CV] degree=1, kernel=poly ...........................................
[CV] .................. degree=1, kernel=poly, score=0.666667 -   0.2s
[CV] degree=1, kernel=poly ...........................................
[CV] .................. degree=1, kernel=poly, score=0.653846 -   0.2s
[CV] degree=2, kernel=poly ...........................................
[CV] .................. degree=1, kernel=poly, score=0.500000 -   0.4s
[CV] degree=2, kernel=poly ...........................................
[CV] .................. degree=1, kernel=poly, score=0.730769 -   0.4s
[CV] degree=2, kernel=poly ...........................................
[CV] .................. degree=1, kernel=poly, score=0.653846 -   0.4s
[CV] degree=2, ke

[Parallel(n_jobs=4)]: Done   3 out of  10 | elapsed:    0.4s remaining:    0.9s


KeyboardInterrupt: 

In [37]:
# clf = ExtraTreesClassifier(n_estimators=250,
#                               random_state=0)
# clf = clf.fit(X, y)
# importances = clf.feature_importances_
# importances
# indices = np.argsort(importances)[::-1]
# #print("Feature ranking:")

# #for f in range(X.shape[1]):
# #    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
# n_features = [10,30,50,70,80,100,1000,2000,13000,18000,10000,20000,3000,30000,4000,500,5000,6000,7000,8000,9000,15000,25000]
# #n_features = [1000]
# n_features.sort()

# for i in n_features:
    
       
#     index=indices[0:i]
#     print(index.shape)
#     features=X[:,index]

#     feature_num=features.shape[1]
#     print(feature_num)
#     lr_all_scores = []
#     lr_f1s = []
#     svm_all_accs = []
#     svm_all_scores = []
#     svm_f1s = []
#     #base_labels = []
#     #for runs in range(1):
#         lr_accuracies = []
#         lr_scores = []
#         svm_accuracies = []
#         svm_scores = []
#         strat_labels = []
        
logistic = linear_model.LogisticRegression(C=1e5)
rbf = svm.SVC(C=10,kernel='linear',gamma=0.01)

skf=RepeatedStratifiedKFold(n_splits=2, n_repeats=1)
lr_all_accs = []
lr_all_scores = []
lr_f1s = []
svm_all_accs = []
svm_all_scores = []
svm_f1s = []

lr_accuracies = []
lr_scores = []
svm_accuracies = []
svm_scores = []
strat_labels = []

        
for train_index, test_index in skf.split(X, y):

        train_data, test_data = X[train_index], X[test_index]
        train_labels, test_labels = y[train_index], y[test_index]
        strat_labels = np.append(strat_labels, test_labels)

        clf = ExtraTreesClassifier(n_estimators=10000, random_state=0)
        clf = clf.fit(train_data, train_labels)
        importances = clf.feature_importances_
        importances
        indices = np.argsort(importances)[::-1]
        #print("Feature ranking:")
        print(train_labels.shape)
        #for f in range(X.shape[1]):
        #    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
        n_features = [10,30,50,70,80,100,1000,2000,13000,18000,10000,20000,3000,30000,4000,500,5000,6000,7000,8000,9000,15000,25000]
        #n_features = [1000]
        n_features.sort()



        for i in n_features:


            index=indices[0:i]
            print(index.shape)
            features=train_data[:,index]

            feature_num=features.shape[1]
            #print(feature_num)


            #train_data, test_data = features[train_index], features[test_index]
            #train_labels, test_labels = y[train_index], y[test_index]


            log = logistic.fit(train_data[:,index], train_labels)
            print(train_data[:,index].shape)
            log_prob = log.decision_function(test_data[:,index])
            log_acc = log.score(test_data[:,index], test_labels)

            #f1 calculation
            y_pred = logistic.predict(test_data[:,index])
            log_f1 = f1_score(test_labels, y_pred)

            lr_accuracies = np.append(lr_accuracies, log_acc)
            lr_scores = np.append(lr_scores, log_prob)
            lr_f1s = np.append(lr_f1s, log_f1)
            print('Logistic Regression Accuracy: %f' % log_acc)

            rbf = rbf.fit(train_data[:,index], train_labels)
            svm_acc = rbf.score(test_data[:,index], test_labels)
            svm_prob = rbf.decision_function(test_data[:,index])

            #f1 calculation
            y_pred = rbf.predict(test_data[:,index])
            svm_f1 = f1_score(test_labels, y_pred)
            print('SVM Accuracy: %f' % svm_acc)
            svm_accuracies = np.append(svm_accuracies, svm_acc)
            svm_scores = np.append(svm_scores, svm_prob)
            svm_f1s = np.append(svm_f1s, svm_f1)
            #print('SVM f1: %f' % svm_f1)


        lr_all_scores = np.append(lr_all_scores, lr_scores)
        lr_fold_avg = np.mean(lr_accuracies)
        lr_all_accs = np.append(lr_all_accs, lr_fold_avg)
        svm_all_scores = np.append(svm_all_scores, svm_scores)
        svm_fold_avg = np.mean(svm_accuracies)
        svm_all_accs = np.append(svm_all_accs, svm_fold_avg)
            #print('Logistic Regression Accuracy: %f' % log_acc_avg)
                    #print('SVM Regression Accuracy: %f' % svm_acc_avg)
        feature_num_all=np.append(feature_num_all,feature_num)
        #print(feature_num_all)
# print(np.mean(lr_all_accs))
        print(np.mean(svm_all_accs))      
        lr_all_feature=np.append(lr_all_feature,np.mean(lr_all_accs))
        svm_all_feature=np.append(svm_all_feature,np.mean(svm_all_accs))
        lr_fls_feature=np.append(lr_fls_feature,np.mean(lr_f1s))
        svm_fls_feature=np.append(svm_fls_feature,np.mean(svm_f1s))
            #base_labels_all = np.append(base_labels_all,strat)
        base_labels = np.append(base_labels, np.mean(strat_labels))

(60, 34727)
[    9    17    18 ..., 34724 34725 34726]


/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:2247: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


(30,)
(10,)
(30, 10)
Logistic Regression Accuracy: 0.500000
SVM Accuracy: 0.500000
(30,)
(30, 30)
Logistic Regression Accuracy: 0.600000
SVM Accuracy: 0.633333
(50,)
(30, 50)
Logistic Regression Accuracy: 0.466667
SVM Accuracy: 0.633333
(70,)
(30, 70)
Logistic Regression Accuracy: 0.500000
SVM Accuracy: 0.633333
(80,)
(30, 80)
Logistic Regression Accuracy: 0.600000
SVM Accuracy: 0.633333
(100,)
(30, 100)
Logistic Regression Accuracy: 0.433333
SVM Accuracy: 0.666667
(500,)
(30, 500)
Logistic Regression Accuracy: 0.300000
SVM Accuracy: 0.633333
(1000,)
(30, 1000)
Logistic Regression Accuracy: 0.400000
SVM Accuracy: 0.633333
(2000,)
(30, 2000)
Logistic Regression Accuracy: 0.333333
SVM Accuracy: 0.633333
(3000,)
(30, 3000)
Logistic Regression Accuracy: 0.233333
SVM Accuracy: 0.633333
(4000,)
(30, 4000)
Logistic Regression Accuracy: 0.300000
SVM Accuracy: 0.633333
(5000,)
(30, 5000)
Logistic Regression Accuracy: 0.266667
SVM Accuracy: 0.633333
(6000,)


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


(30, 6000)
Logistic Regression Accuracy: 0.266667
SVM Accuracy: 0.633333
(7000,)
(30, 7000)
Logistic Regression Accuracy: 0.266667
SVM Accuracy: 0.633333
(8000,)
(30, 8000)
Logistic Regression Accuracy: 0.266667
SVM Accuracy: 0.633333
(9000,)
(30, 9000)
Logistic Regression Accuracy: 0.266667
SVM Accuracy: 0.633333
(10000,)
(30, 10000)
Logistic Regression Accuracy: 0.266667
SVM Accuracy: 0.633333
(13000,)
(30, 13000)
Logistic Regression Accuracy: 0.300000
SVM Accuracy: 0.633333
(15000,)
(30, 15000)
Logistic Regression Accuracy: 0.333333
SVM Accuracy: 0.633333
(18000,)
(30, 18000)
Logistic Regression Accuracy: 0.333333
SVM Accuracy: 0.633333
(20000,)
(30, 20000)
Logistic Regression Accuracy: 0.333333
SVM Accuracy: 0.633333
(25000,)
(30, 25000)
Logistic Regression Accuracy: 0.366667
SVM Accuracy: 0.633333
(30000,)
(30, 30000)
Logistic Regression Accuracy: 0.333333
SVM Accuracy: 0.633333
0.628985507246
(30,)
(10,)
(30, 10)
Logistic Regression Accuracy: 0.466667
SVM Accuracy: 0.566667
(30,)

In [38]:
print(svm_all_feature)

[ 0.62898551  0.59166667]


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot( (feature_num_all), lr_all_feature*100, lw=5, label='Logistic Regression')
plt.plot( (feature_num_all), svm_all_feature*100, lw=5, label='SVM')
plt.xlim([50, 30000])
plt.ylim([0, 100])
plt.xlabel('Number of Features')
plt.ylabel('Accuracy')
plt.title('MC converter vs non-converter')
plt.legend(loc="lower right")
plt.savefig('/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/idp_jiook/data/MCIconvert_combine.pdf')
plt.show()

In [14]:
print("svm accuracy: {}".format(np.mean(svm_acc)))
print("svm f1: {}".format(np.mean(svm_f1s)))
print(svm_f1)


svm accuracy: 0.9473684210526315
svm f1: 0.8695652173913044
0.96


In [1]:
#print(svm_f1s)

# print("LR accuracy Avg: {}".format(np.mean(lr_all_feature)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
# print("LR accuracy Standard Deviation: {}".format(np.std(lr_all_feature)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))

# print("LR f1s Avg : {}".format(np.mean( lr_fls_feature)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
# print("LR f1s Standard Deviation: {}".format(np.std(lr_f1s)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))

# print("SVM Avg: {}".format(np.mean(svm_all_feature)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
# print("SVM Standard Deviation: {}".format(np.std(svm_all_accs)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))

# print("SVM f1s Avg : {}".format(np.mean( svm_fls_feature )),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
# print("SVM f1s Standard Deviation: {}".format(np.std(svm_fls_feature)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
lrindex=np.argmax(lr_all_feature)
# print("LR Maximum feature number is: {}".format(feature_num_all[lrindex]),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
svmindex=np.argmax(svm_all_feature)
# print("SVM Maximum feature is number : {}".format(feature_num_all[svmindex]),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))


# clf = ExtraTreesClassifier(n_estimators=250,
#                               random_state=0)
# clf = clf.fit(X, y)
# importances = clf.feature_importances_
# #importances
# indices = np.argsort(importances)[::-1]
print("Feature ranking:")

for f in range(100):
    print("%d. feature %d %s (%f)" % (f + 1, indices[f],headers[indices[f]],importances[indices[f]]),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMI_rank.txt", "a"))

    
#################################################################################################################    
from sklearn import metrics

max_features = feature_num_all[svmindex]
print(max_features)
lr_all_accs = []
lr_all_scores = []
lr_f1s = []
svm_all_accs = []
svm_all_scores = []
svm_f1s = []
base_labels = []
lr_sensitivity= []
svm_sensitivity=[]
lr_specificity=[]
svm_specificity=[]

index=indices[0:int(max_features)]
features=X[:,index]
feature_num=features.shape[1]
for runs in range(1):
    lr_accuracies = []
    lr_scores = []
    svm_accuracies = []
    svm_scores = []
    strat_labels = []

    logistic = linear_model.LogisticRegression(C=1e5)
    rbf = svm.SVC(C=10,kernel='linear',gamma=0.01)
    
    skf=RepeatedStratifiedKFold(n_splits=10, n_repeats=10)
    for train_index, test_index in skf.split(features, y):
                train_data, test_data = features[train_index], features[test_index]
                train_labels, test_labels = y[train_index], y[test_index]
                #print(train_data.shape)
                strat_labels = np.append(strat_labels, test_labels)

                log = logistic.fit(train_data, train_labels)
                log_prob = log.decision_function(test_data)
                log_acc = log.score(test_data, test_labels)

                #f1 calculation
                y_pred = logistic.predict(test_data)
                log_f1 = f1_score(test_labels, y_pred)
                log_confuse= metrics.confusion_matrix(test_labels, y_pred)
                TP = log_confuse[1, 1]
                TN = log_confuse[0, 0]
                FP = log_confuse[0, 1]
                FN = log_confuse[1, 0]
                log_sen=metrics.recall_score(test_labels, y_pred)
    #             print(TN)
    #             print(FP)
                log_spec=TN / (TN + FP)
    #             print(log_confuse)
    #             print(log_sen)
    #             print(log_spec)

                lr_accuracies = np.append(lr_accuracies, log_acc)
                lr_sensitivity=np.append(lr_sensitivity, log_sen)
                lr_specificity=np.append(lr_specificity, log_spec)

                lr_scores = np.append(lr_scores, log_prob)
                lr_f1s = np.append(lr_f1s, log_f1)

                #print('Logistic Regression Accuracy: %f' % log_acc)

                rbf = rbf.fit(train_data, train_labels)
                svm_acc = rbf.score(test_data, test_labels)
                svm_prob = rbf.decision_function(test_data)

                #f1 calculation
                y_pred = rbf.predict(test_data)
                svm_f1 = f1_score(test_labels, y_pred)
                svm_confuse= metrics.confusion_matrix(test_labels, y_pred)
                TP = log_confuse[1, 1]
                TN = log_confuse[0, 0]
                FP = log_confuse[0, 1]
                FN = log_confuse[1, 0]
                svm_sen=metrics.recall_score(test_labels, y_pred)
                svm_spec=TN / (TN + FP)



                svm_accuracies = np.append(svm_accuracies, log_acc)
                svm_sensitivity=np.append(svm_sensitivity, log_sen)
                svm_specificity=np.append(svm_specificity, log_spec)

                #print('SVM Accuracy: %f' % svm_acc)
                svm_accuracies = np.append(svm_accuracies, svm_acc)
                svm_scores = np.append(svm_scores, svm_prob)
                svm_f1s = np.append(svm_f1s, svm_f1)

    base_labels = np.append(base_labels, strat_labels)
    lr_all_scores = np.append(lr_all_scores, lr_scores)
    lr_fold_avg = np.mean(lr_accuracies)
    lr_all_accs = np.append(lr_all_accs, lr_fold_avg)
    svm_all_scores = np.append(svm_all_scores, svm_scores)
    svm_fold_avg = np.mean(svm_accuracies)
    svm_all_accs = np.append(svm_all_accs, svm_fold_avg)

# feature_num_all=np.append(feature_num_all,feature_num)
# # print(np.mean(lr_all_accs))
print(np.mean(svm_all_accs))      
# lr_all_feature=np.append(lr_all_feature,np.mean(lr_all_accs))
# print(lr_all_feature)
# svm_all_feature=np.append(svm_all_feature,np.mean(svm_all_accs))
# print(np.mean(svm_accuracies))
# lr_fls_feature=np.append(lr_fls_feature,np.mean(lr_f1s))
# svm_fls_feature=np.append(svm_fls_feature,np.mean(svm_f1s))
#base_labels_all = np.append(base_labels_all,strat)
#base_labels = np.append(base_labels, np.mean(strat_labels))

print("LR accuracy Avg: {}".format(np.mean(lr_accuracies)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("LR accuracy Standard Deviation: {}".format(np.std(lr_accuracies)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("LR sensitivity Avg: {}".format(np.mean(lr_sensitivity)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("LR sensitivity Standard Deviation: {}".format(np.std(lr_sensitivity)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("LR specificity Avg: {}".format(np.mean(lr_specificity)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("LR specificity Standard Deviation: {}".format(np.std(lr_specificity)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("LR f1s Avg : {}".format(np.mean(lr_f1s)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("LR f1s Standard Deviation: {}".format(np.std(lr_f1s)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))

print("SVM Avg: {}".format(np.mean(svm_accuracies)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("SVM Standard Deviation: {}".format(np.std(svm_accuracies)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("SVM sensitivity Avg: {}".format(np.mean(svm_sensitivity)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("SVM sensitivity Standard Deviation: {}".format(np.std(svm_sensitivity)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("SVM specificity Avg: {}".format(np.mean(svm_specificity)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("SVM specificity Standard Deviation: {}".format(np.std(svm_specificity)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("SVM f1s Avg : {}".format(np.mean(svm_f1s)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("SVM f1s Standard Deviation: {}".format(np.std(svm_f1s)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
#lrindex=np.argmax(lr_all_feature)
print("LR Maximum feature number is: {}".format(feature_num_all[lrindex]),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
#svmindex=np.argmax(svm_all_feature)
print("SVM Maximum feature is number : {}".format(feature_num_all[svmindex]),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))


###############################################################################################################

from sklearn.metrics import roc_curve, auc,f1_score
import matplotlib.pyplot as plt

fpr = dict()
tpr = dict()
roc_auc = dict()

fpr['lr'], tpr['lr'], _ = roc_curve(base_labels, lr_all_scores)
roc_auc['lr'] = auc(fpr['lr'], tpr['lr'])
#f1['lr']=f1_score(fpr['lr'], tpr['lr'])
fpr['svm'], tpr['svm'], _ = roc_curve(base_labels, svm_all_scores)
roc_auc['svm'] = auc(fpr['svm'], tpr['svm'])
#f1['svm']=f1_score(fpr['svm'], tpr['svm'])
#fpr['gcn'], tpr['gcn'], _ = roc_curve(all_labels, all_scores)
#roc_auc['gcn'] = auc(fpr['gcn'], tpr['gcn'])

plt.figure()
plt.plot(fpr['lr'], tpr['lr'], lw=5, label='Logistic Regression (area = %0.2f)' % roc_auc['lr'] )
#plt.plot(fpr['lr'], tpr['lr'], lw=3, label='Logistic Regression (f1 = %0.2f)' % f1['lr'] )
plt.plot(fpr['svm'], tpr['svm'], lw=5, label='SVM (area = %0.2f)' % roc_auc['svm'] )
#plt.plot(fpr['svm'], tpr['svm'f], lw=3, label='SVM (f1 = %0.2f)' % f1['svm'] )
#plt.plot(fpr['gcn'], tpr['gcn'], lw=3, label='GCN (area = %0.2f)' % roc_auc['gcn'])
plt.plot([0, 1], [0, 1], color='navy', lw=5, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('MCIvsSMI')
plt.legend(loc="lower right") 
#plt.savefig('10x_Combined_ROC.eps')
#plt.savefig('ROC_MCIVsNormal_connectome.eps')
plt.show()

NameError: name 'np' is not defined

In [ ]:
plt.savefig('/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/ROC_MCIvsSMI_connectome.pdf')